In [2]:

import os
import glob
import time
import pandas as pd
from flask import Flask, jsonify
from datetime import datetime, timedelta
from threading import Thread

app = Flask(__name__)
transaction_dir = "./transactions"
reference_file = "./reference/ProductReference.csv"

product_df = pd.read_csv(reference_file)
transactions_df = pd.DataFrame()
merged_df = pd.DataFrame()
loaded_files = set()

def load_transactions():
    global transactions_df, merged_df, loaded_files
    all_files = glob.glob(os.path.join(transaction_dir, "*.csv"))
    new_files = [f for f in all_files if f not in loaded_files]

    if not new_files:
        return

    new_dataframes = [pd.read_csv(f) for f in new_files]
    new_df = pd.concat(new_dataframes, ignore_index=True)
    new_df["transactionDatetime"] = pd.to_datetime(new_df["transactionDatetime"])

    transactions_df = pd.concat([transactions_df, new_df], ignore_index=True)
    merged_df = transactions_df.merge(product_df, on="productId", how="left")
    loaded_files.update(new_files)
    print(f"✅ Loaded {len(new_files)} new file(s).")

def folder_watcher():
    while True:
        load_transactions()
        time.sleep(10)

@app.route("/assignment/transaction/<int:transaction_id>", methods=["GET"])
def get_transaction(transaction_id):
    record = merged_df[merged_df["transactionId"] == transaction_id]
    if record.empty:
        return jsonify({"error": "Transaction not found"}), 404
    result = record.iloc[0]
    return jsonify({
        "transactionId": int(result["transactionId"]),
        "productName": result["productName"],
        "transactionAmount": result["transactionAmount"],
        "transactionDatetime": result["transactionDatetime"].strftime("%Y-%m-%d %H:%M:%S")
    })

@app.route("/assignment/transactionSummaryByProducts/<int:last_n_days>", methods=["GET"])
def transaction_summary_by_products(last_n_days):
    recent_date = datetime.now() - timedelta(days=last_n_days)
    recent_txns = merged_df[merged_df["transactionDatetime"] >= recent_date]
    summary = recent_txns.groupby("productName")["transactionAmount"].sum().reset_index()
    return jsonify({"summary": summary.to_dict(orient="records")})

@app.route("/assignment/transactionSummaryByManufacturingCity/<int:last_n_days>", methods=["GET"])
def transaction_summary_by_city(last_n_days):
    recent_date = datetime.now() - timedelta(days=last_n_days)
    recent_txns = merged_df[merged_df["transactionDatetime"] >= recent_date]
    summary = recent_txns.groupby("productManufacturingCity")["transactionAmount"].sum().reset_index()
    summary.rename(columns={"productManufacturingCity": "cityName"}, inplace=True)
    return jsonify({"summary": summary.to_dict(orient="records")})

def run_app():
    app.run(port=8080)

Thread(target=run_app).start()
Thread(target=folder_watcher, daemon=True).start()
load_transactions()


✅ Loaded 1 new file(s).
✅ Loaded 1 new file(s).
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
